In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from math import atan, pi
import pickle
from os import walk

In [ ]:
# read descriptors
descriptors_directory = 'descriptors/'
_, _, descriptors_names = next(walk(descriptors_directory))
descriptors = []
for descriptor in descriptors_names:
    with open(descriptors_directory+descriptor, 'rb') as file:
        descriptors.append(pickle.load(file))

In [ ]:
# read the image & convert to grayscale
file_name = "test4.jpg"
file_directory = "face_test/"
face= cv2.imread(file_directory+file_name)
face_gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
plt.imshow(face_gray, cmap="gray")

In [ ]:
# detect the face
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
points = face_cascade.detectMultiScale(face_gray, 1.1, 15)
print(points)
# Draw a rectangle around the face
num=0
for (x, y, w, h) in points:
    num+=1
    if(num==len(points)):
        cv2.rectangle(face_gray, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
# Display the resulting image
plt.imshow(face_gray, cmap="gray")

In [ ]:
# normalize the face
for (x, y, w, h) in points:
    face_croped = face_gray[y:y+h, x:x+w].copy()
face_size = 128 # size of the face
face_resized = cv2.resize(face_croped, (face_size,face_size))
plt.imshow(face_resized, cmap="gray")

In [ ]:
window_size = 8 # size of the hog & lbp blocks
w = face_resized.shape[1]
h = face_resized.shape[0]

In [ ]:
# ----LBP----

# duplicate edges
face_resized = cv2.vconcat([face_resized, face_resized[h-1:h, 0:w]]) # +1 row at the end
face_resized = cv2.vconcat([face_resized[0:1, 0:w], face_resized]) # +1 row at the start
face_resized = cv2.hconcat([face_resized, face_resized[0:h+2, w-1:w]]) # +1 column at the end 
face_resized = cv2.hconcat([face_resized[0:h+2, 0:1], face_resized]) # +1 column at the start 

# # calculate lbp
img_lbp = np.zeros((h, w),np.uint8) 
for y in range(-1, h-1):
    for x in range(-1, w-1):
        mini_matrix =  face_resized[(y)+1:(y+3)+1, (x)+1:(x+3)+1].copy()
        seuil = mini_matrix[1][1]
        i1 = 1 if mini_matrix[0][0] >= seuil else 0
        i2 = 2 if mini_matrix[0][1] >= seuil else 0
        i3 = 4 if mini_matrix[0][2] >= seuil else 0
        i4 = 8 if mini_matrix[1][2] >= seuil else 0
        i5 = 16 if mini_matrix[2][2] >= seuil else 0
        i6 = 32 if mini_matrix[2][1] >= seuil else 0
        i7 = 64 if mini_matrix[2][0] >= seuil else 0
        i8 = 128 if mini_matrix[1][1] >= seuil else 0
        somme = i1 + i2 + i3 + i4 + i5 + i6 + i7 + i8
        img_lbp[y+1][x+1] = somme
plt.imshow(img_lbp, cmap='gray')

# calculate lbp histograms
list_hist_lbp = []
for y in range(0, h, window_size):
    for x in range(0, w, window_size):
        block_lbp = img_lbp[y:y+window_size, x:x+window_size].copy()
        list_hist_lbp.append(np.histogram( block_lbp,256,[0,256])[0])

In [ ]:
# ----HOG----

#calculate gx and gy and teta 
Gx = np.zeros((h, w),np.int8) 
Gy = np.zeros((h, w),np.int8) 
teta = np.zeros((h, w),np.int8) 
for y in range(-1, h-1):
    for x in range(-1, w-1):
        mini_matrix =  face_resized[(y)+1:(y+3)+1, (x)+1:(x+3)+1].copy()
        Gx[y+1][x+1] = mini_matrix[1][0] - mini_matrix[1][2]
        Gy[y+1][x+1] = mini_matrix[0][1] - mini_matrix[2][1]
        gx = Gx[y+1][x+1]
        gy = Gy[y+1][x+1]
        if gx == 0:
            if gy == 0:
                teta[y+1][x+1] = -1
            else :
                if gy> 0:
                    teta[y+1][x+1] = 2
                else :
                    teta[y+1][x+1] = 6
        else :
            val = atan(gy/gx)
            degree_val = atan(val)*(180/pi)
            degree_val_corrected = degree_val
            if (gx<0):
                degree_val_corrected += 180
            if (gx>0 and gy<0):
                degree_val_corrected += 360
            if(337.5<=degree_val_corrected or degree_val_corrected<22.5):
                teta[y+1][x+1] = 0
            if(22.5<=degree_val_corrected<67.5):
                teta[y+1][x+1] = 1
            if(67.5<=degree_val_corrected<112.5):
                teta[y+1][x+1] = 2
            if(112.5<=degree_val_corrected<157.5):
                teta[y+1][x+1] = 3
            if(157.5<=degree_val_corrected<202.5):
                teta[y+1][x+1] = 4
            if(202.5<=degree_val_corrected<247.5):
                teta[y+1][x+1] = 5
            if(247.5<=degree_val_corrected<292.5):
                teta[y+1][x+1] = 6
            if(292.5<=degree_val_corrected<337.5):
                teta[y+1][x+1] = 7
plt.figure(figsize=(13,9))
plt.subplot(2,1,1)
plt.title("Gx")
plt.imshow(Gx, cmap="gray")
plt.subplot(2,1,2)
plt.title("Gy")
plt.imshow(Gy, cmap="gray")

# calculate hog histograms
list_hist_hog = []
for y in range(0, h, window_size):
    for x in range(0, w, window_size):
        block_hog = teta[y:y+window_size, x:x+window_size].copy()
        list_hist_hog.append(np.histogram( block_hog,256,[0,256])[0])

In [ ]:
# concatenate hog & lbp
descriptor_test = []
for i in range(0, int((face_size/window_size)**2)):
    descriptor_test.append((list_hist_lbp[i], list_hist_hog[i]))


In [ ]:
is_authorized = False
for i in range(len(descriptors)):
    # compare the two descriptors
    descriptor = descriptors[i]
    mse_lbp = []
    mse_hog = []
    mse = 0;
    for j in range(len(descriptor)):
        mse_lbp.append(((descriptor[j][0]-descriptor_test[j][0])**2).mean())
        mse_hog.append(((descriptor[j][1]-descriptor_test[j][1])**2).mean())
        mse += mse_lbp[j] + mse_hog[j]
    print(mse)
    if(mse<700):
        is_authorized = True
if is_authorized:
    print("YES")
else:
    print("NO")